##fMRI Analysis
1. **preprocess**
2. Individual Level analysis
3. Group Level analysis


###preprocess
1. slice time correction(fsl)
2. motion correction (fsl)
3. artifact detection to identify outliers (fsl)
4. segentation - fsl FAST
5. coregistration with anatomical (fsl)
6. smoothing (fsl)

In [ ]:
!pip install nipype
!pip install nilearn
!pip install bids

##Imports

In [2]:
%matplotlib inline
from os.path import join as opj
import os
import json
from nipype.interfaces.fsl import (BET, ExtractROI, FAST, FLIRT, ImageMaths, MCFLIRT, SliceTimer, Threshold)
from nipype.interfaces.spm import Smooth
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.rapidart import ArtifactDetect
from nipype import Workflow, Node

In [8]:
path = os.listdir('/content/fmri')
path

['sub-xp101-func-sub-xp101_task-fmriNF_bold.json',
 'sub-xp101-func-sub-xp101_task-motorloc_bold.json',
 'sub-xp101-anat-sub-xp101_T1w.nii.gz',
 'sub-xp101-func-sub-xp101_task-fmriNF_bold.nii.gz']

##Parameters

In [8]:
experiment_dir = '/output'
output_dir = 'datasink'
working_dir = 'workingdir'

# list of subject identifiers
subject_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

# list of session identifiers
task_list = ['fingerfootlips']

# Smoothing widths to apply
fwhm = [4, 8]

# TR of functional images
with open('/content/sub-xp101-func-sub-xp101_task-motorloc_bold.json', 'rt') as fp:
    task_info = json.load(fp)
TR = task_info['RepetitionTime']

# Isometric resample of functional images to voxel size (in mm)
iso_size = 4

###Pipeline: Nipype (SPM, FSL, FreeSurfer Interface)

##Layers (Nodes, interfaces) of the main workflow

Option1

In [ ]:
#SLICE TIME CORRECTION
slicetimer = Node(SliceTimer(index_dir=False,
                             interleaved=True,
                             output_type='NIFTI',
                             time_repetition=TR),
                  name="slicetimer")

#MOTION CORRECTION
mcflirt=Node(MCFLIRT(mean_vol=True,
                    save_plots=True,
                     output_type='NIFTI'),
             name="mcflirt")

#ARTIFACT DETECTION
art=Node(ArtifactDetect(norm_threshold=2,
                       zintensity_threshold=2,
                       mask_type='spm_global',
                       parameter_source='FSL',
                       use_differences=[True,False],
                       plot_type='svg'), name='art')


#COREGISTRATION FUNC-ANAT IMAGEs
coreg = Node(FLIRT(dof=6,
                   cost='bbr',
                   schedule='/usr/local/fsl/etc/flirtsch/bbr.sch',
                   output_type='NIFTI'),
             name="coreg")

#SMOOTH

smooth = Node(Smooth(), name="smooth")
smooth.iterables = ("fwhm", fwhm_list)

In [ ]:
Option2

In [ ]:
# ExtractROI - skip dummy scans
extract = Node(ExtractROI(t_min=4, t_size=-1, output_type='NIFTI'),
               name="extract")

# MCFLIRT - motion correction
mcflirt = Node(MCFLIRT(mean_vol=True,
                       save_plots=True,
                       output_type='NIFTI'),
               name="mcflirt")

# SliceTimer - correct for slice wise acquisition
slicetimer = Node(SliceTimer(index_dir=False,
                             interleaved=True,
                             output_type='NIFTI',
                             time_repetition=TR),
                  name="slicetimer")

# Smooth - image smoothing
smooth = Node(Smooth(), name="smooth")
smooth.iterables = ("fwhm", fwhm)

# Artifact Detection - determines outliers in functional images
art = Node(ArtifactDetect(norm_threshold=2,
                          zintensity_threshold=3,
                          mask_type='spm_global',
                          parameter_source='FSL',
                          use_differences=[True, False],
                          plot_type='svg'),
           name="art")


##Coregistration

Option1

In [ ]:
# BET - Skullstrip anatomical Image
# Brain Extraction Tool
# Intensity based
bet_anat = Node(BET(frac=0.5,
                    robust=True,
                    output_type='NIFTI_GZ'),
                name="bet_anat")

# FAST - Image Segmentation
# hidden Markov Random Field (non directed graphs, funtion/clique based)
segmentation = Node(FAST(output_type='NIFTI_GZ'),
                name="segmentation")

# Select WM segmentation file from segmentation output
def get_wm(files):
    return files[-1]

# Threshold - Threshold WM probability image
threshold = Node(Threshold(thresh=0.5,
                           args='-bin',
                           output_type='NIFTI_GZ'),
                name="threshold")

# FLIRT - pre-alignment of functional images to anatomical images
# FMRIB's Linear Image Registration Tool)
coreg_pre = Node(FLIRT(dof=6, output_type='NIFTI_GZ'),
                 name="coreg_pre")

# FLIRT - coregistration of functional images to anatomical images with BBR
# Boundry Based Registration (intensity based)
coreg_bbr = Node(FLIRT(dof=6,
                       cost='bbr',
                       schedule=opj(os.getenv('FSLDIR'),
                                    'etc/flirtsch/bbr.sch'),
                       output_type='NIFTI_GZ'),
                 name="coreg_bbr")

# Apply coregistration warp to functional images
applywarp = Node(FLIRT(interp='spline',
                       apply_isoxfm=iso_size,
                       output_type='NIFTI'),
                 name="applywarp")

# Apply coregistration warp to mean file
applywarp_mean = Node(FLIRT(interp='spline',
                            apply_isoxfm=iso_size,
                            output_type='NIFTI_GZ'),
                 name="applywarp_mean")

# Create a coregistration workflow
coregwf = Workflow(name='coregwf')
coregwf.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the coregistration workflow
coregwf.connect([(bet_anat, segmentation, [('out_file', 'in_files')]),
                 (segmentation, threshold, [(('partial_volume_files', get_wm),
                                             'in_file')]),
                 (bet_anat, coreg_pre, [('out_file', 'reference')]),
                 (threshold, coreg_bbr, [('out_file', 'wm_seg')]),
                 (coreg_pre, coreg_bbr, [('out_matrix_file', 'in_matrix_file')]),
                 (coreg_bbr, applywarp, [('out_matrix_file', 'in_matrix_file')]),
                 (bet_anat, applywarp, [('out_file', 'reference')]),
                 (coreg_bbr, applywarp_mean, [('out_matrix_file', 'in_matrix_file')]),
                 (bet_anat, applywarp_mean, [('out_file', 'reference')]),
                 ])

Option2

In [ ]:
# BET -Skullstrip anatomical Image
bet_anat=Node(BET(frac=0.4,
                 robust=True,
                 output_type='NIFTI_GZ'),
             name="bet_anat")
#SEGMENTATION

# with FAST 
segmentation=Node(FAST(output_type='NIFTI_GZ'),name="segmentation")

#with SPM
#segmentation = Node(NewSegment(tissues=tissues), name='segmentation')


# Select WM segmentation file from segmentation output
# for fast
def get_wm(files):
    return files[-1]

#for spm
#def get_wm(files):
 #   return files[1][0]

#Threshold white matter probablity map 
threshold_WM = Node(Threshold(thresh=0.5,
                              args='-bin',
                              output_type='NIFTI'),
                name="threshold_WM")

#FLIRT - pre-alignement (6 DOF) of functional to anatomical image
coreg_pre=Node(FLIRT(dof=6, output_type='NIFTI_GZ'),name="coreg_pre")

#FLIRT - coregistration of functional to anatomical with BBR
coreg_bbr=Node(FLIRT(dof=6,cost='bbr',
                    schedule=opj(os.getenv('FSLDIR'),'etc/flirtsch/bbr.sch'),
                    output_type='NIFTI_GZ'), name='coreg_bbr')

#APPLY coregistration warp to functional image
applywarp = Node(FLIRT(interp='spline',
                       apply_isoxfm=desired_voxel_iso,
                       output_type='NIFTI'),
                 name="applywarp")

#apply coregistration warp to mean file
applywarp_mean=Node(FLIRT(interp='spline',
                         apply_isoxfm=desired_voxel_iso,
                         output_type='NIFTI_GZ'),
                 name="applywarp_mean")

##Input-Out

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id', 'task_name']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list),
                        ('task_name', task_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
anat_file = opj('derivatives', 'fmriprep', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_t1w_preproc.nii.gz')
func_file = opj('sub-{subject_id}', 'ses-test', 'func',
                'sub-{subject_id}_ses-test_task-{task_name}_bold.nii.gz')

templates = {'anat': anat_file,
             'func': func_file}
selectfiles = Node(SelectFiles(templates,
                               base_directory='/data/ds000114'),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

## Use the following DataSink output substitutions
substitutions = [('_subject_id_', 'sub-'),
                 ('_task_name_', '/task-'),
                 ('_fwhm_', 'fwhm-'),
                 ('_roi', ''),
                 ('_mcf', ''),
                 ('_st', ''),
                 ('_flirt', ''),
                 ('.nii_mean_reg', '_mean'),
                 ('.nii.par', '.par'),
                 ]
subjFolders = [('fwhm-%s/' % f, 'fwhm-%s_' % f) for f in fwhm]
substitutions.extend(subjFolders)
datasink.inputs.substitutions = substitutions

##Workflow

In [ ]:
# Create a preprocessing workflow
preproc = Workflow(name='preproc')
preproc.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow
preproc.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                            ('task_name', 'task_name')]),
                 (selectfiles, extract, [('func', 'in_file')]),
                 (extract, mcflirt, [('roi_file', 'in_file')]),
                 (mcflirt, slicetimer, [('out_file', 'in_file')]),

                 (selectfiles, coregwf, [('anat', 'bet_anat.in_file'),
                                         ('anat', 'coreg_bbr.reference')]),
                 (mcflirt, coregwf, [('mean_img', 'coreg_pre.in_file'),
                                     ('mean_img', 'coreg_bbr.in_file'),
                                     ('mean_img', 'applywarp_mean.in_file')]),
                 (slicetimer, coregwf, [('slice_time_corrected_file', 'applywarp.in_file')]),

                 (coregwf, smooth, [('applywarp.out_file', 'in_files')]),

                 (mcflirt, datasink, [('par_file', 'preproc.@par')]),
                 (smooth, datasink, [('smoothed_files', 'preproc.@smooth')]),
                 (coregwf, datasink, [('applywarp_mean.out_file', 'preproc.@mean')]),

                 (coregwf, art, [('applywarp.out_file', 'realigned_files')]),
                 (mcflirt, art, [('par_file', 'realignment_parameters')]),

                 (coregwf, datasink, [('coreg_bbr.out_matrix_file', 'preproc.@mat_file'),
                                      ('bet_anat.out_file', 'preproc.@brain')]),
                 (art, datasink, [('outlier_files', 'preproc.@outlier_files'),
                                  ('plot_files', 'preproc.@plot_files')]),
                 ])

##Run the Workflow

In [ ]:
preproc.run('MultiProc', plugin_args={'n_procs': 8})